# Q&A con RAG usando modelos locales (sin ejecución de código)
Este tipo de Chatbot es más adecuado para uso general de consultas sobre la base de datos, es decir conocimiento ya existente que no requiera código al ser ejecutado.
Algunas preguntas de ejemplo:
- ¿Cuántas casas a la renta tienen?
- ¿Cuánto cuesta rentar?
- ¿Qué hay más, casas o departamentos en renta?

Tiene la ventaja de que no puede ser usado maliciosamente con intenciones de correr código sin supervisión.

## Datos originales

Escogemos alguna de las dos opciones para procesar inicialmente los datos: convertirlo en documentos y hacer un corte (loaded and split).

La primera opción es cargarlos como un CSV:

In [24]:
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_filepath = './data/listings.csv'
loader = CSVLoader(csv_filepath, csv_args={
    "delimiter": ",",
    "fieldnames": ["id", "Listing Type", "Property Type", "Price in MXN", "Number of Bedrooms", "Number of Bathrooms", "Has Pool", "Has Terrace", "Surface in m2"],
},
)
docs = loader.load()

La segunda opción sería cargarlos como un Dataframe de la librería de pandas (Esta opción es una alternativa a la anterior, escoge una u otra)

In [ ]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader

csv_filepath = './data/listings.csv'
df = pd.read_csv(csv_filepath)
loader = DataFrameLoader(df, page_content_column="id")
docs = loader.load()

## Modelo
Aquí escogemos LLM que deseamos utilizar, en este caso es **llama2**, un modelo Open Source corriendo en nuestro equipo local a través de Ollama.
Este modelo será utilizado en la etapa de 'embedding' para la base de datos vectoriales y como LLM para nuestras queries.
Considero importante utilizar una temperatura de 0 o cercana a él para evitar que el LLM halucine respuestas cuando desconozca los datos.

In [25]:
from langchain_community.chat_models import ChatOllama
# Nombre del modelo
modelo = "llama2"
llm = ChatOllama(
    model=modelo,
    temperature=0,
)

## Base de datos Vectorial

En este caso utilizamos Chromadb como base de datos vectorial para almacenar los datos embebidos y también recuperarlos. El siguiente proceso puede ser tardado para procesarse, sin embargo está configurado actualmente para guardar esta base de datos vectoriales tras ejecutarse la primera vez.

In [26]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Crear el vectorstore
embedding_model = OllamaEmbeddings(model=modelo)
db = Chroma.from_documents(docs, embedding_model, persist_directory="./data", collection_name="chroma_listings")
retriever = db.as_retriever()

Tras haber hecho la celda anterior, es posible cargar esta db desde el directorio guardado, siendo necesario volver a ejecutar la celda de arriba solo cuando se decida cambiar el modelo o se agreguen nuevas entradas.

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Abrir el vectorstore
embedding_model = OllamaEmbeddings(model=modelo)
db = Chroma(persist_directory="./data", embedding_function=embedding_model)
retriever = db.as_retriever()

## Prompt
Necesitamos definir también el template de nuestro prompt, esto nos ayuda a personalizar a la IA con respuestas más acordes a lo que estamos buscando, así como especificarle cómo usar la información previa.

In [27]:
from langchain.prompts import ChatPromptTemplate
template = """Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

## RAG Chain
Esto es la cadena de acciones que nuestro LLM tomará para responder nuestras preguntas, aquí va nuestros

In [28]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
rag_chain = (
    {"context": retriever,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Pruebas de preguntas

In [29]:
query = "Cuanto cuesta en promedio comprar un apartamento?"
rag_chain.invoke(query)

'The average cost of buying an apartment in Mexico can vary depending on several factors such as location, size, and amenities. However, based on the data provided in the documents, the average price of an apartment in Mexico is around 3279050 MXN (approximately $164,000 USD).\n\nThe document with the ID "d6ca294ae021928ff6a899645398670e" provides the most accurate information about the average cost of an apartment in Mexico. According to this document, the average price of an apartment in Mexico is 3279050 MXN (approximately $164,000 USD).\n\nIt\'s important to note that this is just an average and the cost of buying an apartment in Mexico can vary significantly depending on the location, size, and amenities of the property. For example, apartments in popular tourist areas or in major cities like Mexico City may be more expensive than those in smaller towns or rural areas. Additionally, the cost of buying an apartment can also vary depending on the type of ownership, such as freehold 

In [30]:
query = "How many houses do you have?"
rag_chain.invoke(query)

'Based on the provided documents, there are 4 houses for sale.'

In [32]:
query = "Cual es el rango de precio de los departamentos en renta?"
rag_chain.invoke(query)

'The range of prices for apartments in the listings is between 15,200 MXN and 15,200 MXN.'